In [2]:
import pandas as pd

In [2]:
def collect_data(url):
    import requests
    import zipfile
    import io
    response = requests.get(url)
    if response.status_code == 200:
        # Reading zip file from requests response
        zip_file = zipfile.ZipFile(io.BytesIO(response.content))
        # Printing file content in zip
        list_of_files = zip_file.namelist()
        print("ZIP file content:")
        for file_name in list_of_files:
            print(file_name)
        # Extracting file from zip
        zip_file.extractall()
        print("Successfully extracted zip file.")
    else:
        print("Failed to download file. Response:", response.status_code)
    return file_name

In [3]:
# copy url for csv download here
# TODO: the urls need to be updated
urls = [
    "https://www.ebi.ac.uk/chembl/interface_api/delayed_jobs/outputs/DOWNLOAD-TyK57H3Jm1mAnrOx9bQFst7sKvrpnaFiECEPv9KxldU=/DOWNLOAD-TyK57H3Jm1mAnrOx9bQFst7sKvrpnaFiECEPv9KxldU=.zip",
    "https://www.ebi.ac.uk/chembl/interface_api/delayed_jobs/outputs/DOWNLOAD-_1WCrliOUA5F3JVss-saOqsaJ8-pwb9NLfgygYmLUsI=/DOWNLOAD-_1WCrliOUA5F3JVss-saOqsaJ8-pwb9NLfgygYmLUsI=.zip"
]

In [2]:
file_names = []
for url in urls:
    # store file name in file_names
    file_names.append(collect_data(url))

NameError: name 'urls' is not defined

In [2]:
ls

 Volume in drive D is DATA
 Volume Serial Number is 2403-7959

 Directory of d:\PhD Research\MDM2 Project\D1\D1_all_assay_mdm2

29/01/2024  06:24 pm    <DIR>          .
19/01/2024  03:20 pm    <DIR>          ..
29/01/2024  06:24 pm           327,234 data_train.csv
29/01/2024  02:50 pm         2,053,445 DOWNLOAD-_1WCrliOUA5F3JVss-saOqsaJ8-pwb9NLfgygYmLUsI=.csv
29/01/2024  02:50 pm         3,326,441 DOWNLOAD-TyK57H3Jm1mAnrOx9bQFst7sKvrpnaFiECEPv9KxldU=.csv
05/04/2024  03:13 pm            28,702 prep_ml_d1_all_assay_mdm2.ipynb
               4 File(s)      5,735,822 bytes
               2 Dir(s)  192,288,043,008 bytes free


In [20]:
test = pd.read_csv('DOWNLOAD-TyK57H3Jm1mAnrOx9bQFst7sKvrpnaFiECEPv9KxldU=.csv', sep=';')

In [21]:
test.columns

Index(['Molecule ChEMBL ID', 'Molecule Name', 'Molecule Max Phase',
       'Molecular Weight', '#RO5 Violations', 'AlogP', 'Compound Key',
       'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value',
       'Standard Units', 'pChEMBL Value', 'Data Validity Comment', 'Comment',
       'Uo Units', 'Ligand Efficiency BEI', 'Ligand Efficiency LE',
       'Ligand Efficiency LLE', 'Ligand Efficiency SEI', 'Potential Duplicate',
       'Assay ChEMBL ID', 'Assay Description', 'Assay Type', 'BAO Format ID',
       'BAO Label', 'Assay Organism', 'Assay Tissue ChEMBL ID',
       'Assay Tissue Name', 'Assay Cell Type', 'Assay Subcellular Fraction',
       'Assay Parameters', 'Assay Variant Accession', 'Assay Variant Mutation',
       'Target ChEMBL ID', 'Target Name', 'Target Organism', 'Target Type',
       'Document ChEMBL ID', 'Source ID', 'Source Description',
       'Document Journal', 'Document Year', 'Cell ChEMBL ID', 'Properties',
       'Action Type'],
      dtype='object')

In [22]:
test.shape

(2970, 46)

In [21]:
# In case you don't want to redownload the csv files online
# run this block to read csv file names available in current directory
# if you ran previous block above, just skip this block
from pathlib import Path
file_names = []
files = Path('./')
for file in files.iterdir():
    if 'DOWNLOAD' in file.name:
        file_names.append(file.name)

In [22]:
file_names

['DOWNLOAD-TyK57H3Jm1mAnrOx9bQFst7sKvrpnaFiECEPv9KxldU=.csv',
 'DOWNLOAD-_1WCrliOUA5F3JVss-saOqsaJ8-pwb9NLfgygYmLUsI=.csv']

In [23]:
def clean_data(files):
    import pandas as pd
    import numpy as np
    # initiate data_main, main data frame that will be used to aggregate all csv
    # only use chembl ID, smile, and standard value columns
    data_main = pd.DataFrame({'Assay ChEMBL ID':[], 'Molecule ChEMBL ID':[], 'Smiles':[], 'Standard Relation':[], 'Standard Value':[], 'Molecular Weight':[]})
    # for every csv downloaded, do these following tasks
    for file in files:
        # read data
        data = pd.read_csv(file, sep=';')
        # drop rows if Smiles or Standard Value is null
        removed_null = data.dropna(subset=['Smiles', 'Standard Value', 'Standard Relation'])
        # slice data from initial data frame. only use columns in data_main
        sliced = removed_null[data_main.columns]
        # filter the standard relation to only "="
        sliced = sliced[sliced['Standard Relation'].str.contains('=')]
        # filter 100 < MW < 900 (Small molecules)
        sliced = sliced[(sliced['Molecular Weight'] > 100) & (sliced['Molecular Weight'] < 750)]
        # convert IC50 to pIC50
        # convert IC50 from nM to M
        sliced.loc[:, ['Standard Value']] = sliced[['Standard Value']].apply(lambda x: x*10**-9, axis=1)
        # transform to pIC50
        sliced.loc[:, ['Standard Value']] = -np.log10(sliced[['Standard Value']])
        # join data
        data_main = pd.concat([data_main, sliced])
    # drop duplicates according to Smiles column
    # data_main = data_main.drop_duplicates(subset=['Smiles'])
    return data_main[['Assay ChEMBL ID', 'Molecule ChEMBL ID', 'Smiles', 'Standard Value']]
        

In [24]:
# store result of data processing in df_main
df_main = clean_data(file_names)

In [25]:
df_main

,Assay ChEMBL ID,Molecule ChEMBL ID,Smiles,Standard Value
0,CHEMBL3370341,CHEMBL3318761,C[C@]1(CC(=O)O)C[C@H](c2cccc(Cl)c2)[C@@H](c2cc...,10.229148
1,CHEMBL3370341,CHEMBL3318777,CN(C1CC1)S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(CC(=...,9.420216
2,CHEMBL3370341,CHEMBL3318779,C[C@@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(...,9.823909
3,CHEMBL3370341,CHEMBL3318780,C[C@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(C...,8.958607
4,CHEMBL3370341,CHEMBL3318781,CN1CCN(S(=O)(=O)C[C@@H](N2C(=O)[C@@](C)(CC(=O)...,10.130768
...,...,...,...,...
1609,CHEMBL4681440,CHEMBL4744488,CC(=O)N[C@@H]1[C@H](C)N(CC2CC2)[C@@]2(C(=O)Nc3...,6.640165
1610,CHEMBL5110382,CHEMBL5180348,CCOC(=O)C1=NN(c2ccc(Cl)cc2)C2(C(=O)Nc3cc(Cl)cc...,7.150581
1611,CHEMBL5135142,CHEMBL5191434,CCC[C@H]1N(C(=O)c2cnccc2C)CCC[C@@]1(Oc1csc(C(F...,7.468521
1614,CHEMBL5135142,CHEMBL5191746,CCC[C@H]1N(C(=O)c2cnccc2C(F)(F)F)CCC[C@@]1(Oc1...,7.522879


In [26]:
df_main.describe()

,Standard Value
count,2954.000000
mean,7.533789
std,1.711429
min,2.992252
25%,6.238460
50%,7.845577
75%,8.958607
max,10.397940


In [27]:
# save df_main to a csv file
df_main.to_csv('data_MW_threshold.csv')

In [1]:
import pandas as pd

In [31]:
data_new = pd.read_csv('data_cleaned.csv', index_col=0)

In [32]:
data_new.head()

,Assay ChEMBL ID,Molecule ChEMBL ID,Smiles,Standard Value
0,CHEMBL3370341,CHEMBL3318761,C[C@]1(CC(=O)O)C[C@H](c2cccc(Cl)c2)[C@@H](c2cc...,10.229148
1,CHEMBL3370341,CHEMBL3318777,CN(C1CC1)S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(CC(=...,9.420216
2,CHEMBL3370341,CHEMBL3318779,C[C@@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(...,9.823909
3,CHEMBL3370341,CHEMBL3318780,C[C@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(C...,8.958607
4,CHEMBL3370341,CHEMBL3318781,CN1CCN(S(=O)(=O)C[C@@H](N2C(=O)[C@@](C)(CC(=O)...,10.130768


In [4]:
!pip install molvs

In [1]:
from molvs import standardize_smiles

In [2]:
standardize_smiles('FC(F)(F)c1ccc(N2C(=O)[C@](N[C@](C)c3ccccc3)C[C@@]2c2cc(OC)ccc2)cc1')

'COc1cccc([C@]2C[C@@](N[C](C)c3ccccc3)C(=O)N2c2ccc(C(F)(F)F)cc2)c1'

In [34]:
def std_data(data: pd.DataFrame):
    def std_smi(smiles):
        std = standardize_smiles(smiles)
        if std != smiles: print(f"Successfully standardized smiles: {std}")
        return std
    data['Smiles'] = data['Smiles'].apply(std_smi)
    return data

In [35]:
df_std = std_data(data_new)

In [38]:
df_std.head()

,Assay ChEMBL ID,Molecule ChEMBL ID,Smiles,Standard Value
0,CHEMBL3370341,CHEMBL3318761,C[C@]1(CC(=O)O)C[C@H](c2cccc(Cl)c2)[C@@H](c2cc...,10.229148
1,CHEMBL3370341,CHEMBL3318777,CN(C1CC1)S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(CC(=...,9.420216
2,CHEMBL3370341,CHEMBL3318779,C[C@@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(...,9.823909
3,CHEMBL3370341,CHEMBL3318780,C[C@H]1CCCN1S(=O)(=O)C[C@@H](N1C(=O)[C@@](C)(C...,8.958607
4,CHEMBL3370341,CHEMBL3318781,CN1CCN(S(=O)(=O)C[C@@H](N2C(=O)[C@@](C)(CC(=O)...,10.130768


In [39]:
df_std.to_csv('data_std.csv')

In [24]:
pip install pandasql

  Using cached pandasql-0.7.3-py3-none-any.whl
                                              0.0/2.1 MB ? eta -:--:--
     --                                       0.1/2.1 MB 7.0 MB/s eta 0:00:01
     --------                                 0.5/2.1 MB 7.2 MB/s eta 0:00:01
     -------------                            0.7/2.1 MB 6.1 MB/s eta 0:00:01
     --------------------                     1.1/2.1 MB 6.8 MB/s eta 0:00:01
     -------------------------                1.3/2.1 MB 7.0 MB/s eta 0:00:01
     -------------------------------          1.6/2.1 MB 6.9 MB/s eta 0:00:01
     ------------------------------------     1.9/2.1 MB 6.7 MB/s eta 0:00:01
     ---------------------------------------  2.1/2.1 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------- 2.1/2.1 MB 6.3 MB/s eta 0:00:00
                                              0.0/292.8 kB ? eta -:--:--
     -------------------------------------- 292.8/292.8 kB 8.8 MB/s eta 0:00:00
Note: you may need to resta


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
import pandasql

In [41]:
def sorting_data(data):
    from pandasql import sqldf
    df_cols = ['_'.join(col.split(' ')) for col in data.columns]
    df_mod = data.copy()
    df_mod.columns = df_cols
    query = """
    SELECT
    Assay_ChEMBL_ID,
    Smiles,
    MAX(Standard_Value) AS pIC50
    FROM df_mod
    GROUP BY Assay_ChEMBL_ID, Smiles
    ORDER BY COUNT(Assay_ChEMBL_ID) DESC;
    """
    df_sorted = sqldf(query)
    return df_sorted

In [42]:
df_sort = sorting_data(df_std)

In [43]:
df_sort.head(10)

,Assay_ChEMBL_ID,Smiles,pIC50
0,CHEMBL3706024,C[C@H]1CC[C@H](Cn2c(N3CCCC4CCCC43)nc3cc(-c4noc...,9.000000
1,CHEMBL3706331,C[C@H]1CC[C@H](Cn2c(N3CCCC4CCCC43)nc3cc(-c4noc...,9.000000
2,CHEMBL3706024,COCC(C)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3cncc(Cl)c...,9.000000
3,CHEMBL3706024,C[C@H]1CC[C@H](Cn2c(N3CCO[C@H]4CC(F)C[C@@H]43)...,9.000000
4,CHEMBL3706331,COCC(C)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3cncc(Cl)c...,9.000000
5,CHEMBL3706331,C[C@H]1CC[C@H](Cn2c(N3CCO[C@@H]4CC(F)C[C@H]43)...,9.000000
6,CHEMBL1838865,N#Cc1ccc(CN2C(=O)c3cccc(Cl)c3C2(OCC2(CO)CC2)c2...,6.866461
7,CHEMBL1838865,O=C1c2cccc(Cl)c2C(OCC2(CO)CC2)(c2ccc(Cl)cc2)N1...,6.863279
8,CHEMBL1838865,O=C1c2cccc(Cl)c2C(OCC2(CO)CC2)(c2ccc(Cl)cc2)N1...,7.356547
9,CHEMBL3705908,O=C1c2cccc(Cl)c2C(OCC2(CO)CC2)(c2ccc(Cl)cc2)N1...,7.397940


In [44]:
def keep_maxIC50(data):
    print(f'Before dropping duplicates: {data.shape}')
    data = data.drop_duplicates(subset=['Smiles'])
    print(f'After dropping duplicates: {data.shape}')
    return data

In [45]:
df_maxIC50 = keep_maxIC50(df_sort)

Before dropping duplicates: (3726, 3)
After dropping duplicates: (2954, 3)


In [46]:
df_maxIC50.head(30)

,Assay_ChEMBL_ID,Smiles,pIC50
0,CHEMBL3706024,C[C@H]1CC[C@H](Cn2c(N3CCCC4CCCC43)nc3cc(-c4noc...,9.000000
2,CHEMBL3706024,COCC(C)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3cncc(Cl)c...,9.000000
3,CHEMBL3706024,C[C@H]1CC[C@H](Cn2c(N3CCO[C@H]4CC(F)C[C@@H]43)...,9.000000
5,CHEMBL3706331,C[C@H]1CC[C@H](Cn2c(N3CCO[C@@H]4CC(F)C[C@H]43)...,9.000000
6,CHEMBL1838865,N#Cc1ccc(CN2C(=O)c3cccc(Cl)c3C2(OCC2(CO)CC2)c2...,6.866461
7,CHEMBL1838865,O=C1c2cccc(Cl)c2C(OCC2(CO)CC2)(c2ccc(Cl)cc2)N1...,6.863279
8,CHEMBL1838865,O=C1c2cccc(Cl)c2C(OCC2(CO)CC2)(c2ccc(Cl)cc2)N1...,7.356547
10,CHEMBL3706024,CC(F)(c1ccccc1F)c1nc2cc(-c3noc(=O)[nH]3)nc(-c3...,9.000000
11,CHEMBL3706024,CC1CN(c2nc3cc(-c4noc(=O)[nH]4)nc(-c4cncc(Cl)c4...,9.000000
12,CHEMBL3706024,CC1OCCN(c2nc3cc(-c4noc(=O)[nH]4)nc(-c4cncc(Cl)...,9.000000


In [47]:
df_maxIC50.to_csv('data_maxIC50.csv')

In [53]:
df_maxIC50[['Smiles', 'pIC50']].to_csv('data_maxIC50_cleaned.csv', index=False)

In [48]:
df_maxIC50.describe()

,pIC50
count,2954.000000
mean,7.558620
std,1.715466
min,2.992252
25%,6.264604
50%,7.886057
75%,9.000000
max,10.397940
